In [1]:
import pandas as pd
import numpy as np
import time
import os

t = time.localtime()
timestamp = time.strftime('%y%m%d-%H%M', t)

# read excel
df = pd.read_excel('H://Current_Working_Folder//Reports Analytics//202001//202001-t_worc_families-20200316_1131.xlsx')

# assign path and reporitng month
drive = 'H://Current_Working_Folder//Reports Analytics//'
reporting_month = str(df.loc[0,'reporting_month'])

# Filter with column names ending with amt, amount, nbr_month
df_amt = df.filter(regex = 'amt$|amount$|amount$|nbr_month$|nbr_months', axis='columns')

# Describe the filtered dataframe
df_des = df_amt.describe()

# Write the describe to excel
writer = pd.ExcelWriter(drive + reporting_month + '//' + reporting_month + '_' + 'report' + '-' + timestamp + '.xlsx')
#df_des.to_excel(writer, index = False)
df_des.to_excel(writer, sheet_name='family_stat')
writer.save()

In [11]:
df.shape

(2280, 56)

In [2]:
df['subsid_child_care'].value_counts()

No subsidized child care received                                                                                                                 2181
Yes, receives child care funded entirely or in part with Federal funds (e.g., receives TANF, CCDF, SSBG, or other federally funded child care)      99
Name: subsid_child_care, dtype: int64

In [3]:
df['subsid_child_care_amt'].sum()

81730

In [4]:
df['tanf child care_children_covered'].value_counts()

0    2280
Name: tanf child care_children_covered, dtype: int64

In [5]:
df['tanf child care_amount'].sum()

0

In [6]:
cat = df.drop(df_amt, axis='columns')
#cat1 = cat.drop((cat.loc[:,'reporting_month':'stratum']),axis='columns')
cat1 = cat.drop(['id','reporting_month','case_number','county_fips_code','stratum','disposition','worker_id',
                 'case_manager','area','created_by','updated_by','created_at','updated_at','ans_county',
                 'local_office','waiver_grps'], axis='columns')
cat2 = cat1.drop(cat1.filter(regex = '^reason|^other'), axis='columns')

In [7]:
reason_cat = cat1.filter(regex = '^reason', axis='columns')

for c in reason_cat.columns:
    print(reason_cat[c].value_counts())
    print('---')


No     2044
Yes     236
Name: reason for amount of reduction_work requirements sanction, dtype: int64
---
No    2280
Name: reason for amount of reduction_Family Sanction for an Adult with No High School Diploma or Equivalent, dtype: int64
---
No    2280
Name: reason for amount of reduction_Sanction for Teen Parent not Attending School, dtype: int64
---
No     2165
Yes     115
Name: reason for amount of reduction_Non-Cooperation with Child Support, dtype: int64
---
No     2279
Yes       1
Name: reason for amount of reduction_Failure to Comply with an Individual Responsibility Plan, dtype: int64
---
No     2276
Yes       4
Name: reason for amount of reduction_Other Sanction, dtype: int64
---
0    2280
Name: reason for amount of reduction_Recoupment of Prior Overpayment, dtype: int64
---


In [8]:
#((df.filter(regex = '^reason'))).head(20)

In [9]:
for c in cat2.columns:
    print((cat2[c].value_counts().to_frame('count').rename_axis(c)).reset_index())
    print('---')

   category  count
0       Tea   2045
1  WorkPays    235
---
     zip_code  count
0       71854     81
1       72301     70
2       72209     65
3       72401     55
4       72390     39
5       72335     39
6       72114     38
7       72143     38
8       72956     37
9       72204     37
10      72764     36
11      72315     35
12      72076     33
13      72360     32
14      71603     32
15      71601     31
16      72206     30
17      72342     30
18      72904     30
19      72901     29
20      72117     28
21      72450     27
22      71701     26
23      71730     25
24      72032     23
25      72703     22
26      72701     22
27      71913     22
28      71953     21
29      72601     21
..        ...    ...
334     72579      1
335     72567      1
336     72565      1
337     72523      1
338     72863      1
339     74960      1
340     72935      1
341     72941      1
342     72052      1
343     72036      1
344     72030      1
345     72020      1
346     72010  

In [10]:
worc_fam = df
#fam_type = worc_fam['family_type'].value_counts()
fam_type = worc_fam.groupby(['category','family_type']).agg({'case_number' : 'nunique'}).rename({'case_number': 'case_count'}, axis='columns')#.reset_index()
fam_type

case_count
category family_type                                                   
Tea      Family included only in overall work participat...         906
         Family with no work-eligible individual                   1084
         Not found                                                    1
         Two-Parent Family included in both the overall ...          54
WorkPays Family included only in overall work participat...         224
         Two-Parent Family included in both the overall ...          11

In [6]:
exempt = df.groupby(['zip_code']).agg({'exempt_time_limit':'value_counts'}).sort_values(['exempt_time_limit'], ascending=False)
exempt

exempt_time_limit
zip_code exempt_time_limit                                                    
71854    Family is not exempt from Federal time limit.                      51
72401    Family is not exempt from Federal time limit.                      39
72301    Family is not exempt from Federal time limit.                      36
          Yes, family is exempt from accrual of months a...                 33
72209     Yes, family is exempt from accrual of months a...                 32
72764     Yes, family is exempt from accrual of months a...                 30
72209    Family is not exempt from Federal time limit.                      30
72390    Family is not exempt from Federal time limit.                      30
72401     Yes, family is exempt from accrual of months a...                 30
72335    Family is not exempt from Federal time limit.                      29
71854     Yes, family is exempt from accrual of months a...                 27
72204    Family is not exempt from Federal time limit.                      27
72114     Yes, family is exempt from accrual of months a...                 25
72360    Family is not exempt from Federal time limit.                      24
72450    Family is not exempt from Federal time limit.                      23
71603    Family is not exempt from Federal time limit.                      22
72143    Family is not exempt from Federal time limit.                      22
72076     Yes, family is exempt from accrual of months a...                 21
         Family is not exempt from Federal time limit.                      20
72315    Family is not exempt from Federal time limit.                      20
72204     Yes, family is exempt from accrual of months a...                 20
72956     Yes, family is exempt from accrual of months a...                 19
72904     Yes, family is exempt from accrual of months a...                 19
72117     Yes, family is exempt from accrual of months a...                 19
72390     Yes, family is exempt from accrual of months a...                 18
71601     Yes, family is exempt from accrual of months a...                 18
71701    Family is not exempt from Federal time limit.                      18
72764    Family is not exempt from Federal time limit.                      18
72901    Family is not exempt from Federal time limit.                      18
72342    Family is not exempt from Federal time limit.                      18
...                                                                        ...
72135     Yes, family is exempt from accrual of months a...                  1
72132    Family is not exempt from Federal time limit.                       1
72131     Yes, family is exempt from accrual of months a...                  1
72130     Yes, family is exempt from accrual of months a...                  1
72125     Yes, family is exempt from accrual of months a...                  1
72124     Yes, family is exempt from accrual of months a...                  1
72121     Yes, family is exempt from accrual of months a...                  1
72106    Family is not exempt from Federal time limit.                       1
          Yes, family is exempt from accrual of months a...                  1
72173    Family is not exempt from Federal time limit.                       1
72175     Yes, family is exempt from accrual of months a...                  1
72176    Family is not exempt from Federal time limit.                       1
72303    Family is not exempt from Federal time limit.                       1
72348     Yes, family is exempt from accrual of months a...                  1
72333     Yes, family is exempt from accrual of months a...                  1
72331     Yes, family is exempt from accrual of months a...                  1
72327    Family is not exempt from Federal time limit.                       1
72321    Family is not exempt from Federal time limit.                       1
72319     Yes, family is exempt 

In [7]:
exempt = df.groupby(df['zip_code']==72209).agg({'exempt_time_limit':'value_counts'})
exempt

exempt_time_limit
zip_code exempt_time_limit                                                    
False    Family is not exempt from Federal time limit.                    1185
          Yes, family is exempt from accrual of months a...               1096
True      Yes, family is exempt from accrual of months a...                 32
         Family is not exempt from Federal time limit.                      30

In [11]:
program.shape

(393, 3)

In [17]:
program = (df.groupby(['zip_code','local_office'])['category']).size().sort_values(ascending=False).to_frame('count').reset_index()
#program.describe()
size = df.groupby('zip_code').agg({'category':'value_counts'}).rename({'category':'count'},axis = 'columns').reset_index()
#p = (program >19).values.sum()
#program.shape
 

size.nlargest(40, 'count')
#program.nsmallest(40, program)#
##one = program[program['count']==1]#.values.sum()
##one
#program
#size.sort_values('count', ascending=False)
#wp_by_zip = size.loc[size['category']==84].sort_values('count', ascending=False)#['count'].max()
#wp_by_zip.nlargest(10, 'count')



In [34]:

import openpyxl

t = program.query('count == 1')
t

os.chdir('H://Current_Working_Folder//Reports Analytics//201907//')
file_name = 'test.xlsx'
writer = pd.ExcelWriter(file_name, engine='openpyxl')

if os.path.exists(file_name):
    book = openpyxl.load_workbook(file_name)
    writer.book = book

t.to_excel(writer, sheet_name='test', index=False)
writer.save()
writer.close()

In [221]:
non_ar = df.loc[df['zip_code']< 70000, ('category','case_number','local_office','case_manager')]#.reset_index()

non_ar
#a = frame.loc[frame['Doc'] == 'Inv', 'Rep']

,category,case_number,local_office,case_manager
1685,20,7972501,22,0
1705,20,7999474,13,0
1733,20,7977886,17,0
1864,20,7974676,3,1001
1866,20,8001263,12,1003


In [38]:
#df.groupby(['zip_code','food_stamps']).agg({'category':'value_counts'})
df.groupby(['new_applicant','zip_code']).size().to_frame('Total ')

Total 
new_applicant                     zip_code        
Not newly-approved application    39180          1
                                  53210          1
                                  65201          1
                                  65791          1
                                  71446          1
                                  71601         33
                                  71602         13
                                  71603         37
                                  71611          4
                                  71631          1
                                  71635         20
                                  71638          9
                                  71639          5
                                  71640         11
                                  71643          2
                                  71646          6
                                  71647          5
                                  71652          1
                                  71653          9
                                  71654         12
                                  71655         12
                                  71657          3
                                  71660          2
                                  71661          1
                                  71664          1
                                  71665          4
                                  71667          7
                                  71671         15
                                  71674          2
                                  71675          2
...                                            ...
Yes, a newly-approved application 71953          1
                                  72082          1
                                  72118          1
                                  72120          1
                                  72201          1
                                  72204          1
                                  72210          1
                                  72315          1
                                  72331          1
                                  72342          1
                                  72358          1
                                  72360          3
                                  72365          2
                                  72401          2
                                  72404          1
                                  72450          1
                                  72455          2
                                  72601          1
                                  72706          1
                                  72712          1
                                  72714          1
                                  72715          1
                                  72761          1
                                  72762          1
                                  72764          1
                                  72863          1
                                  72901          1
                                  72903          2
                                  72936          1
                                  72944          1

[390 rows x 1 columns]